In [1]:
from pathlib import Path
import os

yr = 2023
d = 4

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import re


def format_input(inp):

  def extract_numbers(l):
    l = l.split(':')[1]
    target_nums = [n.strip() for n in l.split('|')[0].split(' ')]
    scratched_nums = [n.strip() for n in l.split('|')[1].split(' ')]
    target_nums = [int(x) for x in target_nums if x != '']
    scratched_nums = [int(x) for x in scratched_nums if x != '']
    return target_nums, scratched_nums

  formatted_input = {}
  for l in inp.split('\n'):
    id_extraction_re = re.compile(r'Card\s+(?P<id>\d+):')
    id_results = id_extraction_re.match(l)
    cur_id = int(id_results['id'])
    formatted_input[cur_id] = {}
    t, s = extract_numbers(l)
    formatted_input[cur_id]['target'] = t
    formatted_input[cur_id]['scratched'] = s
  return formatted_input



In [3]:
def get_scratchard_values(formatted_input):
  card_values = []
  for id in formatted_input.keys():
    winners = len(
                  set(formatted_input[id]['target'])
                  .intersection(set(formatted_input[id]['scratched']))
                  )
    card_values.append(2**(winners-1) if winners > 0 else 0)
  return card_values

In [4]:
def get_recursive_card_counts(formatted_input):
  # Get the number of winners each card has
  cards_2_winners = {}
  for id in formatted_input.keys():
      winners = len(
                set(formatted_input[id]['target'])
                .intersection(set(formatted_input[id]['scratched']))
                )
      cards_2_winners[id] = winners

  def calculate_cards_to_extra_cards(c2w):
    '''
    Starting at the last card, figure out how
    many "EXTRA" cards you get from owning a
    copy of each card
    '''
    c2ec = {}
    for id in reversed(sorted(list(c2w.keys()))):
      if id == max(c2w.keys()):
        c2ec[id] = c2w[id]
      else:
        cur_extras = c2w[id]
        for i in range(1,cur_extras+1):
          cur_extras += c2ec[id+i]
        c2ec[id] = cur_extras
    return c2ec

  # This is the number of extra cards received for having each card
  extra_counts = calculate_cards_to_extra_cards(cards_2_winners)

  # The total number of cards we end up with is the original number
  # of cards plus the number of extra cards that each card yields
  return len(extra_counts) + sum(extra_counts.values())




In [5]:
import time


t = time.time()

formatted_input = format_input(inp)

print(sum(get_scratchard_values(formatted_input)))
print(get_recursive_card_counts(formatted_input))

print("\nRUNTIME: ", time.time()-t)

21213
8549735

RUNTIME:  0.003989219665527344
